# Tensorflow Exercise: Keras Sequential vs. Functional APIs

In this exercise, we will practice using the two APIs that Keras provides for building deep learning models: the Keras Sequential and Functional APIs.

If you need to reference the syntax of either model, see the Keras documentation pages on the [Sequential](https://keras.io/getting-started/sequential-model-guide/) and [Functional](https://keras.io/getting-started/functional-api-guide/) APIs.

## Part 1: Sequential Voting

For our toy problem, we will use the following data:

In [1]:
np.random.seed(42)

NameError: name 'np' is not defined

In [ ]:
import numpy as np
X = np.random.randint(0, 2, size = (1000, 9))
Y = np.where(np.mean(X, axis = 1) > 0.5, 1, 0)

**Questions:**
1. What does it mean that the elements of Y represent a "majority vote" on X?

Y[i] is 1 if the mean of X[i] is > 1 and 0 otherwise, it is the same as saying lets Y[i] be the majority vote of X[i], because if the majority is 0 then the mean is gonna be below 0.5 and if the majority is 1 the mean will be over 0.5.

2. We want to learn how to predict elements of Y from rows of X. Build a Keras Sequential model *model* with one Dense layer (with activation = 'sigmoid') that can be fit on X and Y. Check that the input and output shapes of the model (*model.input_shape* and *model.output_shape*) match the shapes of X and Y.

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


model = Sequential()
model.add(Dense(units=1, activation='sigmoid', input_shape=(9,)))

print("Input shape:", model.input_shape)
print("Output shape:", model.output_shape)


3. Compile the model with 'mean_squared_error' loss, 'rmsprop' optimizer, and *metrics = 'accuracy'*, and fit it to X and Y with *validation_split = 0.2*. You may choose any values for *epochs* and *batch_size* that result in the model learning well.

In [ ]:
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])

model.fit(X, Y, epochs=10, batch_size=32, validation_split=0.2)

4. Once the model has been fit, examine the values of *model.get_weights()*. How do you interpret these values?

In [ ]:
model.get_weights()

The first array represents the weights of the Dense layer.
The second array represents the bias of the Dense layer.

## Part 2: Making it Functional

Now we will practice using Keras's Functional API by rewriting the above model.

**Questions:**

5. Create a model *model2* identical to the above model, but using the Keras Functional API. The model should include an *Input(shape=...)* layer from keras.layers and should use *Model(inputs = ..., outputs = ...)* from keras.models. Fit this model, verify that it produces the same results, and compare the outputs of *.summary()* and *.get_weights()* on *model* and *model2*.

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:

input_layer = Input(shape=(9,))
dense_layer = Dense(units=1, activation='sigmoid')(input_layer)
model2 = Model(inputs=input_layer, outputs=dense_layer)

model2.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])
model2.fit(X, Y, epochs=10, batch_size=32, validation_split=0.2)

print("Summary of model:")
model.summary()

print("\nSummary of model2:")
model2.summary()

print("\nWeights of model:")
print(model.get_weights())

print("\nWeights of model2:")
print(model2.get_weights())

## Part 3: Identifying identical distributions

The previous problem had a nice solution using the Keras Sequential API, but sometimes we will need the Functional API to build more complicated networks. Let's try to learn a slightly more complicated pattern that will be solved more naturally with the Functional API.  In this example, we'll see use of multiple inputs and a shared layer.

Let's generate another dataset:

In [ ]:
M1 = np.array([np.random.choice([-1, 1]) for i in range(10000)])
M2 = np.array([np.random.choice([-1, 1]) for i in range(10000)])
S1 = np.stack([
    np.random.normal(m, 1, size = 5)
    for m in M1
])
S2 = np.stack([
    np.random.normal(m, 1, size = 5)
    for m in M2
])
labels = np.where(M1 == M2, 1, 0)

Every row of S1 and S2 is a sample of 5 elements from a distribution with mean either -1 or 1, and the labels in *label* represent whether the given samples are drawn from the same distribution (0: different distributions, 1: same distribution).

We want to train a model to learn how to predict if the two given samples of 5 data points are drawn from the same distribution, i.e. whether they have the same mean.

**Questions:**

6. Create a Functional model using the following architecture:
  * Two Input layers *inp1* and *inp2*, each taking input of dimension 5
  * A Dense layer *shared_dense* with output dimension 1 and tanh activation function, shared between the input layers. (Define the Dense layer as *shared_dense = Dense(...)* and then set *x1 = shared_dense(inp1)* and *x2 = shared_dense(inp2)*). This means that the same weights will be applied to both inputs.
  * Concatenate the outputs of the dense layers together with *merged = concatenate([x1, x2])*
  * A Dense layer with output dimension 2 and tanh activation function, applied to *merged*
  * A Dense layer with output dimension 1 and sigmoid activation function, applied to the output of the previous dense layer
  * Finally, define the model as *func_model = Model(inputs = ..., outputs = ...)* for the proper inputs and outputs parameters.

In [ ]:
from tensorflow.keras.layers import concatenate

In [ ]:
inp1 = Input(shape=(5,))
inp2 = Input(shape=(5,))

shared_dense = Dense(units=1, activation='tanh')

x1 = shared_dense(inp1)
x2 = shared_dense(inp2)

merged = concatenate([x1, x2])
dense_2 = Dense(units=2, activation='tanh')(merged)
output = Dense(units=1, activation='sigmoid')(dense_2)

func_model = Model(inputs=[inp1, inp2], outputs=output)

7. Examine the input and output shapes of *func_model* and verify that they match *S1*, *S2*, and *labels*.

In [ ]:
func_model.summary()

print("Shape of S1:", S1.shape)
print("Shape of S2:", S2.shape)

print("Shape of labels:", labels.shape)


8. Compile *func_model* with optimiser *sgd*, *binary_crossentropy* loss, and *metrics = 'accuracy'* and fit to *[S1, S2]* and *labels* with *validation_split = 0.2*. Hint: you can use *epochs = 10* and *batch_size = 4* if you are unsure of good values for these hyperparameters. What is the final accuracy that this model achieves? Note: You may have to re-run your code multiple times for the model to learn well, due to randomness. You should get accuracy above 0.95.

In [ ]:
from tensorflow.keras.optimizers import SGD

func_model.compile(optimizer=SGD(), loss='binary_crossentropy', metrics=['accuracy'])
history = func_model.fit([S1, S2], labels, epochs=10, batch_size=4, validation_split=0.2)
final_accuracy = history.history['val_accuracy'][-1]
print("Final Accuracy:", final_accuracy)

9. Compare the predicted probabilities based of 2 inputs coming from the same distribution to the actual labels.  Do these make sense?

**Bonus:** Can you interpret the weights in *func_model.get_weights()*?

In [ ]:
predicted_probabilities = func_model.predict([S1, S2])

for i in range(len(labels)):
    if labels[i] == 1:
        print("Predicted Probability:", predicted_probabilities[i][0], "Actual Label:", labels[i])


The predicted probabilities make sense. Since the actual labels are all 1, and the predicted probabilities are all close to or above 0.95, it suggests that the model is confident that the inputs are from the same distribution.